<a href="https://colab.research.google.com/github/aleksandr-del/kaggle-competitions/blob/main/titanic_competition_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Решение задачи [Titanik](https://www.kaggle.com/c/titanic) на Kaggle

План работы такой:
- скачать и загрузить тренировочные и тестовые данные
- предобработать тренировочные данные
- обучить модель на тренировочных данных
- предобработать тестовые данные
- получить предсказания модели на тестовых данных
- записать предскзания модели в csv файл 
- отправить csv файл на Kaggle 

## Загрузка данных

Импортируем библиотеки:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline # конвейер подготовки данных
from sklearn.pipeline import Pipeline # конвйер алгоритмов
from sklearn.impute import SimpleImputer # пропуски заполняются медианными значениями
from sklearn.preprocessing import OneHotEncoder, StandardScaler # onehot encoding и стандартизации
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

Загрузим обучающие данные:

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI Start/Datasets/ Titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Разделим обучающую выборку на признаки и целевую переменную:

In [ ]:
y = df['Survived']
X = df.drop(columns='Survived')

Выведим общую информацию о признаках:

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB


Видим, что пропуски есть в колонках Age, Cabin, Embarked, а категориальными являются переменные Name, Sex, Ticket, Cabin и Embarked. 

## Предобработка обучающей выборки

### Удаление ненужных признаков

Во-первых, удалим колонки PassengerId, Name, Ticket, Cabin. Эти колонки не несут полезной информации для модели.

In [ ]:
X = X.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

Посмотрим, какие колонки у нас остались с пропусками и какие остались категориальными:

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


### Заполнение пропусков

Заполним пропуски в колонках Age и Embarked.

В колонке Age заполним пропуски средним значением возраста пассажиров из тренировочных данных:

In [ ]:
X['Age'] = X['Age'].fillna(X['Age'].mean())

В колонке Embarked заполним пропуски самым часто встречающимся значением. Выведем, сколько раз каждое значение встречается в колонке:

In [ ]:
X['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

Видим, что самое частое значение — S. Им и заполним пропуски в колонке Embarked:

In [ ]:
X['Embarked'] = X['Embarked'].fillna('S')

Убедимся, что пропусков не осталось:

In [ ]:
X.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

### Перевод категориальных признаков в числовые

У нас есть два категориальных признака: Sex и Embarked. Переведем их в числовые с помощью LabelEncoder из Sklearn:

In [ ]:
le_sex = LabelEncoder()

In [ ]:
X['Sex'] = le_sex.fit_transform(X['Sex'])

In [ ]:
le_embarked = LabelEncoder()

In [ ]:
X['Embarked'] = le_embarked.fit_transform(X['Embarked'])

Убедимся, что в данных не осталось категориальных колонок:

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


## Обучаем модель

Давайте обучим две модели машинного обучения — логистическую регрессию и KNN, — и с помощью кросс-валидации оценим их качество. Затем получим с их помощью овтеты на тестовой выборке и отправим ответы на Kaggle.

Проведем кросс-валидацю логистической регрессии:

In [ ]:
lr = LogisticRegression()

In [ ]:
lr_scores = cross_val_score(lr, X, y, scoring='accuracy', cv=5)

In [ ]:
print(f'Значения метрики accuracy на кросс-валидации: {lr_scores}.')

Значения метрики accuracy на кросс-валидации: [0.78212291 0.78089888 0.78089888 0.76966292 0.82022472].


In [ ]:
print(f'Среднее значение метрики accuracy на кросс-валидации: {np.mean(lr_scores)}.')

Среднее значение метрики accuracy на кросс-валидации: 0.786761659657272.


Проведем кросс-валидацию KNN с количеством ближайших соседей 5:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn_scores = cross_val_score(knn, X, y, scoring='accuracy', cv=5)

In [ ]:
print(f'Значения метрики accuracy на кросс-валидации: {knn_scores}.')

Значения метрики accuracy на кросс-валидации: [0.65363128 0.65168539 0.70786517 0.74157303 0.71910112].


In [ ]:
print(f'Среднее значение метрики accuracy на кросс-валидации: {np.mean(knn_scores)}.')

Среднее значение метрики accuracy на кросс-валидации: 0.694771200803465.


Обучим обе модели на всем тренировочном датасете:

In [ ]:
lr.fit(X, y)
knn.fit(X, y)

KNeighborsClassifier()

## Предообработка тестовой выборки

Загрузим тестовую выборку:

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI Start/Datasets/ Titanic/test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Удалим колонки PassengerId, Name, Ticket, Cabin:

In [ ]:
test = test.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

Посмотрим, какие колонки у нас остались с пропусками и какие остались категориальными:

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       332 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 23.0+ KB


Заполним пропуски в колонках Age и Fare.

В колонке Age заполним пропуски средним значением возраста пассажиров из тренировочных данных:

In [ ]:
test['Age'] = test['Age'].fillna(X['Age'].mean())

В колонке Fare заполним пропуски средним значением возраста пассажиров из тренировочных данных:

In [ ]:
test['Fare'] = test['Fare'].fillna(X['Fare'].mean())

Убедимся, что пропусков не осталось:

In [ ]:
test.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

У нас есть два категориальных признака: Sex и Embarked. Переведем их в числовые с помощью LabelEncoder из Sklearn:

In [ ]:
test['Sex'] = le_sex.transform(test['Sex'])

In [ ]:
test['Embarked'] = le_embarked.transform(test['Embarked'])

Убедимся, что в данных не осталось категориальных колонок:

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    float64
 6   Embarked  418 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 23.0 KB


## Предсказания моделей на тестовой выборке

Получим предсказания моделей логистической регрессии и KNN на тестовых данных:

In [ ]:
y_pred_lr = lr.predict(test)
y_pred_knn = knn.predict(test)

## Запись предсказаний в файл для посылки на Kaggle

В записи предсказаний моделей в файлы для посылки на Kaggle нам поможет файл gender_submission.csv. Откроем его:

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI Start/Datasets/ Titanic/gender_submission.csv')

In [ ]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


Файл gender_submission.csv показывает, в каком формате нужно отправлять ответы на Kaggle. Видим, что формат — это для каждого id пассажира из столбца PassengerId заполнить его значение в колонке Survived – 0 или 1.

Давайте просто заменим значения в колонке Survived таблицы gender_submission.csv на наши предсказания моделей:

In [ ]:
submission['Survived'] = y_pred_lr

Записываем таблицу в файл для отправки на Kaggle:

In [ ]:
submission.to_csv('submission.csv', index=None)

## Конвейер

Загрузим обучающую и тестовую выборки выборку:

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI Start/Datasets/ Titanic/train.csv')

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI Start/Datasets/ Titanic/test.csv')

Удалим ненужные признаки:

In [ ]:
train = train.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

In [ ]:
test = test.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

Поделим обучающую выборку на признаки и целевую переменную:

In [ ]:
 X_train = train.drop(columns='Survived')
 y_train = train['Survived']

Создадим список категориальных признаков:

In [ ]:
cat_features = X_train.select_dtypes(include=object).columns.tolist()
cat_features

['Sex', 'Embarked']

Создадим список количественных признаков:

In [ ]:
num_features = X_train.select_dtypes(exclude=object).columns.tolist()
num_features

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

Создадим конвейер подготовки количественных признаков:

In [ ]:
num_transformer = make_pipeline(SimpleImputer(strategy='median'),
                                StandardScaler())

Создадим конвейер подготовки качественных признаков:

In [ ]:
cat_transformer = make_pipeline(SimpleImputer(strategy='most_frequent'),
                                OneHotEncoder(handle_unknown='ignore',
                                              sparse=False))

Создадим preprocessor для подготовки всех данных:

In [ ]:
preprocessor = ColumnTransformer([('num', num_transformer, num_features),
                                  ('cat', cat_transformer, cat_features)],
                                 verbose_feature_names_out=False)

Создадим финальный конвейер для подготовки данных и обучения алгоритма:

In [ ]:
alg = Pipeline(steps=[('preprocessor', preprocessor), # подготовка данных
                      ('estimator', RandomForestClassifier())]) # алгоритм

Создадим сетку гиперпараметров для алгоритма Случайного леса:

In [ ]:
# название этпапа подготовки__гиперпараметр: [сетка (список) параметров]

parameters = {
    'estimator__n_estimators': [100, 300, 500, 1000],
    'estimator__max_depth': np.arange(1, 13),
    'estimator__max_features': ['auto', 'sqrt', 'log2', None]#,
    #'estimator__max_depth': np.arange(1, 16)
}

In [ ]:
gs = GridSearchCV(alg, # выбор алгоритма или пайплайна (подготовка + алгоритм)
                  parameters, # сетка гиперпараметров
                  cv=5, # количество фолдов кросс-валидации
                  scoring='accuracy', # метрика качества
                  verbose=2, # отображать процесс перебора гиперпараметров
                  n_jobs=2 ) # параллелизм вычислений, соответствует количеству ядер (в colab их 2)

Перебираем гиперпараметры:

In [ ]:
gs.fit(X_train, y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['Pclass',
                                                                          'Age',
                                                                          'SibSp',
                                                                          'Parch',
                                                               

Лучшие гиперпараметры:

In [ ]:
gs.best_params_

{'estimator__max_depth': 7,
 'estimator__max_features': None,
 'estimator__n_estimators': 300}

Лучшая метрика accuracy модели:

In [ ]:
gs.best_score_

0.8361810306948717

В записи предсказаний моделей в файлы для посылки на Kaggle нам поможет файл gender_submission.csv. Откроем его:

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI Start/Datasets/ Titanic/gender_submission.csv')
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


Предсказания модели на тестовой выборке:

In [ ]:
y_pred_rf = gs.best_estimator_.predict(test)

In [ ]:
submission['Survived'] = y_pred_rf
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


Записываем таблицу в файл для отправки на Kaggle:

In [ ]:
submission.to_csv('submission.csv', index=None)